# KOSWAT Handleiding

Deze handleiding is geschreven voor gebruikers van KOSWAT.

Het doel van deze handleiding is om gebruikers een beeld te geven hoe invoerbestanden (.ini) worden gemaakt en wat ze precies inhouden.
In deze invoerbestanden wordt namelijk het hele project gedefinieerd in 9 tabbladen.

Deze tabbladen worden 1-voor-1 behandeld

In [ ]:
import numpy as np
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
from itables import init_notebook_mode
init_notebook_mode(all_interactive=True)
from ipyleaflet import Map, GeoData, basemaps, LayersControl, LegendControl, DrawControl
import geopandas as gpd
import leafmap
import re
import json
import os
from scripts.plotprofile_org import plot_profile, kies_dijksectie
from scripts.maak_selectie_kaart import maak_interactieve_kaart
import warnings
from shapely.geometry import shape
import ipywidgets as widgets
from IPython.display import display, clear_output
from ipyfilechooser import FileChooser
from pathlib import Path
warnings.filterwarnings("ignore")
from ipywidgets import interact, FloatSlider


In [ ]:
# --- JSON chooser ---
chooser_ini = FileChooser(filter_pattern="*.json")

# --- Confirm button ---
confirm_button = widgets.Button(
    description="Laad configuratiebestand",
    button_style="success",
    icon="file-text"
)
output_paths = widgets.Output()

# --- Button click handler ---
def on_confirm_click(b):
    with output_paths:
        clear_output()
        try:
            # Get selected file
            pad_ini = Path(chooser_ini.selected)
            if not pad_ini:
                print("Selecteer eerst een JSON-configuratiebestand.")
                return

            parent_dir = pad_ini.parent
            # Load JSON file
            with open(pad_ini, "r", encoding="utf-8") as f:
                config = json.load(f)

            # Save globally
            globals()['pad_ini'] = pad_ini
            globals()['parent_dir'] = parent_dir
            globals()['config'] = config

            # Extract paths from config
            rel_vakken = config["analyse"]["dijksectie_ligging"]
            rel_profielen = config["analyse"]["dijksectie_invoer"]
            rel_selectie = config["analyse"]["dijksecties_selectie"]
            rel_scenarios = config["analyse"]["scenario_invoer"]
            # Convert to absolute paths
            pad_vakken = parent_dir / rel_vakken
            pad_profielen = parent_dir / rel_profielen
            pad_scenarios = parent_dir / rel_scenarios
            selectie_pad = parent_dir / rel_selectie


            globals()['pad_vakken'] = pad_vakken
            globals()['pad_profielen'] = pad_profielen
            globals()['pad_scenarios'] = pad_scenarios
            globals()['selectie_pad'] = selectie_pad

            # Show results
            print("Configuratiebestand succesvol geladen!\n")
            print(f"pad_ini: {pad_ini}")
            print(f"pad_vakken: {pad_vakken}")
            print(f"pad_profielen: {pad_profielen}")

        except FileNotFoundError:
            print("Het geselecteerde bestand bestaat niet.")
        except json.JSONDecodeError as e:
            print(f"Ongeldig JSON-bestand: {e}")
        except KeyError as e:
            print(f"Vereiste sleutel ontbreekt in de configuratie: {e}")
        except Exception as e:
            print(f"Er is een fout opgetreden: {e}")

# --- Connect and display ---
confirm_button.on_click(on_confirm_click)

display(
    widgets.VBox([
        widgets.HTML("<b>Selecteer het JSON-configuratiebestand:</b>"),
        chooser_ini,
        confirm_button,
        output_paths
    ])
)



In [ ]:
# --- Button & Output ---
knop_ini = widgets.Button(
    description="Lees JSON-configuratie!",
    button_style="info",
    icon="file-text"
)
output_ini = widgets.Output()

# --- Button click function ---
def on_click_ini(b):
    with output_ini:
        clear_output()
        try:
            # Load JSON file
            with open(pad_ini, "r", encoding="utf-8") as f:
                config = json.load(f)

            # Store globally so other cells can access it
            globals()["config"] = config

            # Count sections
            num_sections = len(config.keys())

            print(f"Configuratiebestand succesvol ingelezen!")
            print(f"Aantal secties gevonden: {num_sections}")
            print(f"Secties: {', '.join(config.keys())}\n")

            # Print a preview of each section
            for section, params in config.items():
                print(f"[{section}]")
                for key, value in params.items():
                    print(f"  {key}: {value}")
                print()  # blank line between sections

        except FileNotFoundError:
            print("Het opgegeven pad naar het JSON-bestand bestaat niet.")
        except json.JSONDecodeError as e:
            print(f"Ongeldig JSON-formaat: {e}")
        except Exception as e:
            print(f"Er is een fout opgetreden: {e}")

# --- Connect button ---
knop_ini.on_click(on_click_ini)

# --- Display button and output ---
display(knop_ini, output_ini)

# Analyse

In het eerste deel van het invoerbestand worden de dijksecties ingeladen en geselecteerd.  
De selectie van dijksecties wordt gedefinieerd.  
Per dijksectie wordt het profiel (doorsnede gegeven).  
De verschillende mate van dijkversterking (per dijksectie) worden gedefinieerd als scenario's.

Daarnaast wordt verder relevante informatie gegeven:

- De eenheidsprijzen  
- De omgevingsdatabases  
- De uitvoerfolder

### Dijksecties

Dijksecties worden gegeven als een shapefile (lijnen) en bevatten het traject, subtraject en lengte van elk dijkvak als informatie in de tabel  
Deze informatie wordt hieronder getoond als tabel en als interactieve kaart

In [ ]:
from itables import show

# --- Button & Output ---
knop_vakken_tabel = widgets.Button(
    description="Toon dijkvakken-tabel",
    button_style="info",
    icon="table"
)
output_vakken = widgets.Output()

# --- Button click function ---
def on_click_vakken(b):
    with output_vakken:
        clear_output()
        try:
            dijksecties = gpd.read_file(pad_vakken)
            selectie_vakken = np.loadtxt(selectie_pad, dtype=str)
            selectie_vakken = np.array(selectie_vakken, dtype=str)
            globals()["dijksecties"] = dijksecties  # store globally
            globals()["selectie_vakken"] = selectie_vakken  # store globally

            print(f"{len(dijksecties)} dijkvakken geladen uit {pad_vakken}")
            print("Hieronder staat de interactieve tabel:")

            # Display the interactive itables table
            show(dijksecties)

        except FileNotFoundError:
            print("Het pad naar het vakkenbestand is ongeldig of het bestand bestaat niet.")
        except Exception as e:
            print(f"Er is een fout opgetreden: {e}")

# --- Connect button ---
knop_vakken_tabel.on_click(on_click_vakken)

# --- Display button and output ---
display(knop_vakken_tabel, output_vakken)

In [ ]:
# --- Button & Output ---
knop_kaart_dijksecties = widgets.Button(
    description="Toon kaart van dijksecties",
    button_style="success",
    icon="map"
)
output_kaart = widgets.Output()

# --- Button click function ---
def on_click_kaart(b):
    with output_kaart:
        clear_output()
        try:
            if "dijksecties" not in globals():
                print("Laad eerst de dijkvakken-tabel via de vorige knop.")
                return

            # Zorg dat selectie_vakken bestaat
            if "selectie_vakken" not in globals():
                print("Laad eerst de selectie_vakken.")
                return


            # Maak de kaart
            m = leafmap.Map(center=[52.6, 6], zoom=7)
            m.layout.height = "700px"

            # Split de dijksecties op basis van selectie
            geselecteerde = dijksecties[dijksecties["Dijksectie"].isin(selectie_vakken)]
            overige = dijksecties[~dijksecties["Dijksectie"].isin(selectie_vakken)]

            # Voeg lagen toe
            m.add_gdf(
                overige,
                layer_name="Overige dijksecties",
                style={
                    "color": "red",
                    "weight": 3,
                    "fillOpacity": 0.0,
                },
            )

            m.add_gdf(
                geselecteerde,
                layer_name="Geselecteerde dijksecties",
                style={
                    "color": "yellow",
                    "weight": 4,
                    "fillOpacity": 0.1,
                },
            )

            display(m)
            print(f"{len(dijksecties)} dijksecties weergegeven op de kaart.")
            print(f"{len(geselecteerde)} geselecteerde dijksecties gemarkeerd in geel.")

        except Exception as e:
            print(f"Er is een fout opgetreden: {e}")
# --- Connect button --- 
knop_kaart_dijksecties.on_click(on_click_kaart) 
# # --- Display button and output --- 
display(knop_kaart_dijksecties, output_kaart)



Het bestand 'dijksecties_selectie.txt' bevat de selectie van dijksecties die worden beschouwd in het KOSWAT project.    
Dit is een tekstbestand met een eigen regel voor een geselecteerde dijksectie.  

In de volgende interactieve kaart kun je een zelf een selectie maken van dijksecties voor het KOSWAT project


In [ ]:
# --- Button & Output ---
knop_selectie_kaart = widgets.Button(
    description="Selecteer dijksecties op de kaart",
    button_style="success",
    icon="map-marker"
)
output_selectie_kaart = widgets.Output()

# --- Button click function ---
def on_click_selectie_kaart(b):
    with output_selectie_kaart:
        clear_output()
        try:
            global selectie, m   # Make map + selection available to later buttons

            print("Interactieve kaart geopend — gebruik de knop links om dijksecties te selecteren.")
            print("Als je klaar bent, klik dan op 'Laat mijn selectie zien'.")

            # Create map and selection container
            m, selectie = maak_interactieve_kaart(pad_vakken)
            m.layout.height = "700px"
            display(m)

        except NameError:
            print("Zorg dat je eerst het pad naar de dijkvakken hebt ingevuld!")
        except Exception as e:
            print(f"Er is een fout opgetreden: {e}")

# --- Connect button ---
knop_selectie_kaart.on_click(on_click_selectie_kaart)

# --- Display button + output area ---
display(knop_selectie_kaart, output_selectie_kaart)

In [ ]:
# Voeg een knop toe om de selectie te laten zien
knop_selectie = widgets.Button(description="Laat mijn selectie zien", button_style='success')
output_selectie  = widgets.Output()

def on_button_click_selectie(b):
    with output_selectie:
        clear_output()
        if selectie["gdf"].empty:
            print("Maak eerst een selectie van dijksecties")
        else:
            display(selectie["gdf"])
            # Extract the Dijksectie column as strings (just in case)
            nieuwe_selectie = selectie['gdf']['Dijksectie'].astype(str).tolist()

            # Write to file, one value per line (overwrite existing)
            with open(selectie_pad, "w") as f:
                for val in nieuwe_selectie:
                    f.write(f"{val}\n")

            print(f"Bestand '{selectie_pad}' is bijgewerkt met {len(nieuwe_selectie)} dijksecties.")

knop_selectie.on_click(on_button_click_selectie)
display(knop_selectie, output_selectie)

# Eenheidsprijzen

In het volgende keuze menu kun je een keuze maken tussen twee verschillende eenheidsprijzen. 

Kies er eentje en druk op de knop om je keuze te bevestigen

In [ ]:
# Create UI elements
choice_widget = widgets.RadioButtons(
    options=[
        ("2013", "eenheidsprijzen_2013.json"),
        ("2023", "eenheidsprijzen_2023.json")
    ],
    description="Kies het jaar waar uit de eenheidsprijzen moeten komen:",
)

confirm_button = widgets.Button(
    description="Bevestig keuze",
    button_style="primary"
)

output_prijzen = widgets.Output()

# Global (or state object) to store user's choice
user_choice = {"value": None}

def on_confirm(b):
    with output_prijzen:
        clear_output()

        # Write the chosen filename directly into config
        config["analyse"]["eenheidsprijzen"] = choice_widget.value
        print("Je hebt gekozen voor :", config["analyse"]["eenheidsprijzen"])

confirm_button.on_click(on_confirm)

# Display UI
display(choice_widget, confirm_button, output_prijzen)

# Buffer-en-overgangszone

KOSWAT heeft als instelling standaard een bufferzone en een overgangszone. Deze twee instellingen houden het volgende in:

-  Bufferzone: Dit is de buffer om de gebouwen heen voor een constructieve maatregel
-  Overgangszone: Dit is de afstand tussen twee constructieve maatregelen vanaf waar ze doorgetrokken worden

In het volgende interactieve figuur kun je zelf de buffer-en-overgangszone aanpassen om een beeld te krijgen hoe deze twee variabelen invloed hebben op het versterkingsprofiel

In [ ]:
from matplotlib.patches import Rectangle

def update_buffer_transition_plot(buffer, transition):
    fig, ax = plt.subplots(figsize=(10, 10))
    ax.set_xlim(0, 100)
    ax.set_ylim(-300, 700)
    ax.set_yticks([])  # Remove y-axis ticks

    ax.axvspan(0, 30, color='lightblue', alpha=0.5)
    ax.axvspan(30, 100, color='peru')
    ax.axvline(30, color='green', linewidth=5, label='Dijk')

    xwand1 = [40, 40]
    ywand1 = [(225 - buffer), (275 + buffer)]
    yovergang1 = [(225- buffer - transition), (275 + buffer + transition)]

    xwand2 = [40, 40]
    ywand2 = [(425- buffer), (475 + buffer)]
    yovergang2 = [(425- buffer - transition), (475 + buffer + transition)]

    xwand3 = [40, 40]
    ywand3 = [(-25 - buffer), (25 + buffer)]
    yovergang3 = [(-25 - buffer - transition), (25 + buffer + transition)]

    ax.plot([50, 50], [(225 - buffer), (275 + buffer)], linestyle=':', color='red', label='Bufferzone')
    ax.plot([50, 50], [(425 - buffer), (475 + buffer)], linestyle=':', color='red')
    ax.plot([50, 50], [(-25 - buffer), (25 + buffer)], linestyle=':', color='red')

    if min(yovergang2) < max(yovergang1) and min(yovergang1) < max(yovergang3):
        yovergang = [min(yovergang3), max(yovergang2)]
        ywand = [min(ywand3), max(ywand2)]

        ax.plot(xwand1, yovergang, linestyle='--', color='red', label='Overgangszone')
        ax.plot(xwand2, ywand, color='black', label='Stabiliteitswand')

    elif min(yovergang2) < max(yovergang1):
        yovergang = [min(yovergang1), max(yovergang2)]
        ywand = [min(ywand1), max(ywand2)]
        ax.plot(xwand1, yovergang, linestyle='--', color='red', label='Overgangszone')
        ax.plot(xwand2, ywand, color='black', label='Stabiliteitswand')
        
        ax.plot(xwand3, yovergang3, linestyle='--', color='red')
        ax.plot(xwand3, ywand3, color='black', label='Stabiliteitswand')
    
    elif min(yovergang2) > max(yovergang1):
        ax.plot(xwand1, yovergang1, linestyle='--', color='red', label='Overgangszone')
        ax.plot(xwand2, yovergang2, linestyle='--', color='red')
        ax.plot(xwand3, yovergang3, linestyle='--', color='red')

        ax.plot(xwand1, ywand1, color='black', label='Stabiliteitswand')
        ax.plot(xwand2, ywand2, color='black')
        ax.plot(xwand3, ywand3, color='black')

    # Create and add filled rectangle
    rect1 = Rectangle(
        (50, 225),        # bottom-left corner (x, y)
        5,               # width  = 75 - 50
        50,               # height = 275 - 225
        facecolor='grey',
        edgecolor='black',
        alpha=0.6
    )

    # Create and add filled rectangle
    rect2 = Rectangle(
        (50, 425),        # bottom-left corner (x, y)
        5,               # width  = 75 - 50
        50,               # height = 275 - 225
        facecolor='grey',
        edgecolor='black',
        alpha=0.6
    )

    # Create and add filled rectangle
    rect3 = Rectangle(
        (50, -25),        # bottom-left corner (x, y)
        5,               # width  = 75 - 50
        50,               # height = 275 - 225
        facecolor='grey',
        edgecolor='black',
        alpha=0.6
    )

    ax.add_patch(rect1)
    ax.add_patch(rect2)
    ax.add_patch(rect3)

    ax.legend()
    plt.show()

buffer0 = 10
transition0 = 50
interact(
    update_buffer_transition_plot,
    buffer=FloatSlider(min=0.5 * buffer0, max=2 * buffer0, step=0.1, value=buffer0, description="Buffer"),
    transition=FloatSlider(min=0.5 * transition0, max=2 * transition0, step=0.1, value=transition0, description="Overgang")
);



# Selecteer de te gebruiken maatregelen

Wil je nou geen grondmaatregelen of alleen maar kistdammen? Dan kun je dat hier invullen!

In [ ]:
checkboxes = {
    "grondmaatregel": widgets.Checkbox(
        value=True,
        description="Gebruik grondmaatregelen"
    ),
    "verticalepipingoplossing": widgets.Checkbox(
        value=False,
        description="Gebruik een verticale piping oplossing"
    ),
    "kwelscherm": widgets.Checkbox(
        value=True,
        description="Gebruik kwelschermen"
    ),
    "stabiliteitswand": widgets.Checkbox(
        value=True,
        description="Gebruik stabiliteitswanden"
    ),
    "kistdam": widgets.Checkbox(
        value=True,
        description="Gebruik kistdammen"
    ),
}

maatregel_button = widgets.Button(
    description="Bevestig selectie",
    button_style="primary"
)

output_maatregel = widgets.Output()

def on_confirm_maatregel(b):
    with output_maatregel:
        clear_output()

        for key, checkbox in checkboxes.items():
            config[key]["actief"] = checkbox.value

        print("Configuratie bijgewerkt:")
        for key in checkboxes:
            print(f"{key}: {config[key]["actief"]}")

maatregel_button.on_click(on_confirm_maatregel)

display(
    widgets.VBox([
        widgets.HTML("<b>Kies versterkingsmaatregelen</b>"),
        widgets.VBox(list(checkboxes.values())),
        maatregel_button,
        output_maatregel
    ])
)

# Dijkprofiel en versterkingsopgaven
Per dijksectie wordt de dijkdoorsnede gegeven door een .txt bestand in de folder dijksectie_invoer. 
Hieronder kun je zelf het profiel schetsen van de geselecteerde dijkvakken

De volgende invoerparameters zijn de versterkingsscenario's. Deze beschrijven de versterkingsopgave Dat ziet er als volgt uit   

<!-- ![Voorbeeld scenarios](Figuren\Scenarios.png)            -->

Per scenario zijn er drie versterkingsparameters:   
- dH: De hoogte opgave
- dS: De stabiliteits opgave 
- dP: De piping opgave

Deze drie opgave zijn ook geschetst in de schematisatie. Gebruik de sliders om deze aan te passen om te kijken wat voor effect deze hebben




In [ ]:
button_profiel = widgets.Button(
    description="Toon profiel",
    button_style="success",  # 'success', 'info', 'warning', 'danger'
    icon='play'              # optional icon
)
output_profiel = widgets.Output()

def on_button_click_profiel(b):
    with output_profiel:
        clear_output()
        try:
            selectie_dijksectie = selectie['gdf']
            kies_dijksectie(selectie_dijksectie, plot_profile, pad_profielen, pad_scenarios)
        except NameError:
            print("Maak eerst een selectie van dijksecties")

button_profiel.on_click(on_button_click_profiel)

display(button_profiel, output_profiel)

# Overzicht KOSWAT berekening
Druk op deze knop om een overzicht te krijgen van je aangepaste veranderingen



In [ ]:
button_overzicht = widgets.Button(
    description="Toon veranderingen",
    button_style="success",  # 'success', 'info', 'warning', 'danger'
    icon='play'              # optional icon
)
output_overzicht = widgets.Output()

def on_button_click_overzicht(b):
    with output_overzicht:
        clear_output()
        try:
            selectie_dijksectie = selectie['gdf']
            selectie_dijksectie = selectie_dijksectie.drop(columns="geometry")
            print("Je hebt de volgende dijksecties geselecteerd:")
            for idx, row in selectie_dijksectie.iterrows():
                print(row)

            print("Je hebt gekozen voor :", config["analyse"]["eenheidsprijzen"])

            print("De volgende maatregeltypes worden doorgerekend: ")
            if config["grondmaatregel"]["actief"] == True:
                print('Maatregel in grond')

            if config["verticalepipingoplossing"]["actief"] == True:
                print('Verticale piping oplossingen')
            
            if config["kwelscherm"]["actief"] == True:
                print('Kwelschermen')

            if config["stabiliteitswand"]["actief"] == True:
                print('Stabiliteitswanden')
                
            if config["kistdam"]["actief"] == True:
                print('Kistdammen')

            json_destination = parent_dir.joinpath("aangepast_project.json")
            # Write the JSON file
            with json_destination.open("w", encoding="utf-8") as f:
                json.dump(config, f, ensure_ascii=False, indent=2)
        except NameError:
            print("Maak eerst een selectie van dijksecties")

button_overzicht.on_click(on_button_click_overzicht)

display(button_overzicht, output_overzicht)

# Draai KOSWAT

Draai hier je zelf gemaakt leipe KOSWAT project

In [ ]:
from koswat.koswat_handler import KoswatHandler

# --- timestamp parser ---
def parse_timestamp(line):
    idx = line.find("M")
    if idx == -1:
        return None
    timestamp_str = line[:idx+1].strip()
    fmt = "%Y-%m-%d %I:%M:%S %p"
    return datetime.strptime(timestamp_str, fmt)

# Output widget
output = widgets.Output()

# --- main analysis function ---
def run_koswat_analysis():
    output_folder = parent_dir.joinpath("Output")
    json_project = parent_dir.joinpath("aangepast_project.json")
    log_file = output_folder.joinpath("koswat.log")

    if log_file.exists():
        log_file.unlink()

    with KoswatHandler(output_folder) as _handler:
        _handler.run_analysis(json_project)

# --- combined callback ---
def on_button_click(b):
    with output:
        clear_output()
        print("Berekening gestart...")

        run_koswat_analysis()

        print("Berekening voltooid, timestamp wordt geanalyseerd...")

        # Parse timestamps
        log_file = parent_dir.joinpath("Output").joinpath("koswat.log")

        if not log_file.exists():
            print("Geen logbestand gevonden")
            return

        with open(log_file, "r", encoding="utf-8") as f:
            lines = [line.strip() for line in f.readlines() if line.strip()]

        if not lines:
            print("Logbestand is leeg")
            return
        
        t0 = parse_timestamp(lines[0])
        t_end = parse_timestamp(lines[-1])

        if not t0 or not t_end:
            print("Kon geen timestamps vinden in het logbestand")
            return

        duration = t_end - t0
        minutes, seconds = divmod(duration.total_seconds(), 60)

        if "Finalized Koswat" in lines[-1]:
            print(f"Berekeningen voltooid in {int(minutes)} minuten en {int(seconds)} seconden")
        else:
            print("Er is iets misgegaan, probeer het opnieuw")

# --- button ---
run_and_check_button = widgets.Button(
    description="Draai KOSWAT",
    button_style="primary"
)

run_and_check_button.on_click(on_button_click)

display(run_and_check_button, output)

# Resultaten!

In [ ]:
def toon_resultaten(filename):
    uitvoer_folder = parent_dir.joinpath("uitvoer")

    vak_folder = uitvoer_folder.joinpath(f'dike_{filename}')
    vak_kosten = vak_folder.joinpath('scenario_scenario')
    kosten_file = vak_kosten.joinpath('summary_costs.csv')
    kosten_tabel = pd.read_csv(kosten_file, delimiter=';', engine="python", usecols=range(6))
    kosten_tabel = kosten_tabel.set_index("Profile type")
    kosten_tabel = kosten_tabel.loc[['Cost per km incl surtax (Euro/km)', 'Total measure meters', 'Total measure cost incl surtax', 'Infrastructure cost incl surtax' , 'Total cost incl surtax']]
    kosten_tabel = kosten_tabel.rename(index={
    'Cost per km incl surtax (Euro/km)': 'Kosten per kilometer [M€/ km]',
    'Total measure meters': 'Totaal toegepaste lengte van de maatregel',
    'Total measure cost incl surtax': 'Totale kosten maatregel incl. BTW [M€]',
    'Infrastructure cost incl surtax': 'Kosten infrastructuur incl. BTW [M€]',
    'Total cost incl surtax': 'Totale kosten incl. BTW [M€]'
})
    rows_to_scale = [
        'Kosten per kilometer [M€/ km]',
        'Totale kosten maatregel incl. BTW [M€]',
        'Kosten infrastructuur incl. BTW [M€]',
        'Totale kosten incl. BTW [M€]',
    ]

    kosten_tabel.loc[rows_to_scale, :] = kosten_tabel.loc[rows_to_scale, :] / 1000000
    display(kosten_tabel)

    location_folder = vak_kosten.joinpath('summary_locations')
    shapefile_versterkingen = location_folder.joinpath('summary_locations_new.shp')
    versterkingen = gpd.read_file(shapefile_versterkingen)
    versterkingen = versterkingen.to_crs(epsg=4326)
    
    # Define color map for your maatregel types
    color_map = {
        'Grondmaatregel profiel': 'blue',
        'Verticale piping oplossing': 'green',
        'Kistdam': 'red',
        'Stabiliteitswand': 'orange'
    }

    # Maak de kaart
    m = leafmap.Map(center=[52.6, 6], zoom=3)
    m.layout.width = "1300px"
    m.layout.height = "800px"

    all_layers = []
    for maatregel, color in color_map.items():
        subset_maatregel = versterkingen[versterkingen['maatregel'] == maatregel]
        geo_data_maatregel = GeoData(
            geo_dataframe=subset_maatregel,
            style={
                'color': 'black',
                'fillColor': color,
                'fillOpacity': 0.7,
                'weight': 1
            },
            name=f'{maatregel}'
    )
        m.add_layer(geo_data_maatregel)
        all_layers.append(geo_data_maatregel)

    m.add(LayersControl())
    legend = LegendControl(color_map, title="Legenda", position="bottomright")
    m.add(legend)

    m.zoom_to_gdf(versterkingen)
    
    display(m)
    
def kies_resultaten(df, func):
    # Create dropdown with Dijksectie values
    dropdown = widgets.Dropdown(
        options=sorted(df['Dijksectie'].unique()),
        description='Dijksectie:',
        layout=widgets.Layout(width='300px')
    )

    # Create an output area for the plot
    out = widgets.Output()

    def on_change_resultaten(change):
        if change['type'] == 'change' and change['name'] == 'value':
            with out:
                out.clear_output(wait=True)  # 🧹 clear old plot
                plt.close('all')              # (optional) close any lingering figures
                func(change['new'])

    dropdown.observe(on_change_resultaten)

    display(dropdown, out)

button_resultaten = widgets.Button(
    description="Toon resultaten",
    button_style="success",  # 'success', 'info', 'warning', 'danger'
    icon='play'              # optional icon
)
output_resultaten = widgets.Output()

def on_button_click_resultaten(b):
    with output_resultaten:
        clear_output()
        try:
            selectie_dijksectie = selectie['gdf']
            kies_resultaten(selectie_dijksectie, toon_resultaten)
        except NameError:
            print("Maak eerst een selectie van dijksecties")

button_resultaten.on_click(on_button_click_resultaten)

display(button_resultaten, output_resultaten)

In [ ]:
# kosten_tabel = pd.read_csv(r"C:\Users\stouten\GitHub\Repos\Koswat\basic_case\Voorbeeld_Project\uitvoer\dike_48-1-1-A-1-F\scenario_scenario\summary_costs.csv",
#                             delimiter=';', engine="python", usecols=range(6))
# kosten_tabel = kosten_tabel.set_index("Profile type")
# kosten_tabel = kosten_tabel.loc[['Cost per km incl surtax (Euro/km)', 'Total measure meters', 'Total measure cost incl surtax', 'Infrastructure cost incl surtax' , 'Total cost incl surtax']]
# display(kosten_tabel)